# Imports

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense, Lambda, Input, Flatten, Layer
from tensorflow.keras.models import Model
from tensorflow.python.keras.utils.vis_utils import plot_model


# Custom Layer

In [2]:

class MySimpleDense(Layer):
    def __init__(self, neurons=32, activation=None):
        super(MySimpleDense, self).__init__()
        self.units = neurons
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        w_int = tf.random_normal_initializer()
        self.w = tf.Variable(initial_value=w_int(shape=(input_shape[-1], self.units), dtype=float),
                             trainable=True, name='kernel')

        b_int = tf.zeros_initializer()
        self.b = tf.Variable(name='bias', trainable=True,
                             initial_value=b_int(shape=(self.units,)), dtype=float)

    def call(self, inputs, **kwargs):
        return self.activation(tf.matmul(inputs, self.w) + self.b) # this does w.T * X


# Data 1

In [4]:
xs = np.array([1,2,3,4,5,6,7,8,9], dtype=float)
ys = 2*xs - 1


model = tf.keras.Sequential([
                             MySimpleDense(1)
])
model.compile(loss = 'mse',
              optimizer='sgd')

model.fit(xs, ys, epochs=500, verbose=0)
model.predict([10])

In [5]:
model.variables

array([[18.903664]], dtype=float32)

[<tf.Variable 'my_simple_dense/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[1.9740175]], dtype=float32)>,
 <tf.Variable 'my_simple_dense/bias:0' shape=(1,) dtype=float32, numpy=array([-0.8365115], dtype=float32)>]

# Data 2 , Mnist

In [7]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    MySimpleDense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

11493376/11490434 [==============================] - 0s 0us/step
Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4783 - accuracy: 0.8599
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1572 - accuracy: 0.9532
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1091 - accuracy: 0.9677
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0903 - accuracy: 0.9733
Epoch 5/5
313/313 [==============================] - 1s 1ms/step - loss: 0.0793 - accuracy: 0.9765


[0.07933908700942993, 0.9764999747276306]